# **Aula 1: Analisando sentimentos**

## **Classificando sentimentos**

Modelo: [DistilBERT base uncased finetuned SST-2](https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english)

In [1]:
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
os.environ["USE_MPS"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["PYTORCH_MPS_DEVICE_OVERRIDE"] = "0"

import torch
torch.set_default_device("cpu")

# Simple device configuration
device = torch.device("cpu")
print(f"Using device: {device}")
print(f"PyTorch version: {torch.__version__}")

from transformers import pipeline


Using device: cpu
PyTorch version: 2.7.0.dev20250126


In [2]:
classificador_sentimento = pipeline('sentiment-analysis', 
                                   model='distilbert-base-uncased-finetuned-sst-2-english', 
                                   device=device)

In [3]:
classificador_sentimento('I love this product')

[{'label': 'POSITIVE', 'score': 0.9998788833618164}]

In [4]:
classificador_sentimento('I am totally disappointed with this product')

[{'label': 'NEGATIVE', 'score': 0.9998010993003845}]

## **Selecionando o modelo**

Modelo: [pysentimiento/bertweet-pt-sentiment](https://huggingface.co/pysentimiento/bertweet-pt-sentiment)

In [5]:
from pysentimiento import create_analyzer

In [6]:
def create_cpu_model():
    # Create model with explicit CPU device
    model = create_analyzer(
        task='sentiment',
        lang='pt',
        device="cpu"
    )
    # Force model to CPU
    model.model.to("cpu")
    model.tokenizer.model_max_length = 512
    return model

modelo_analise_sentimento = create_cpu_model()

In [7]:
modelo_analise_sentimento = create_analyzer(task='sentiment', 
                                           lang='pt',
                                           device="cpu")

In [8]:
modelo_analise_sentimento.predict('''
A fritadeira é sensacional, muito além do que imaginava. É linda, super funcional e muito fácil de manusear.
Fácil de limpar e potente. Super recomendo.
''')

AnalyzerOutput(output=POS, probas={POS: 0.989, NEU: 0.009, NEG: 0.002})

In [9]:
modelo_analise_sentimento.predict('''
Após poucos meses de uso a carcaça de aço escovado começou a oxidar,
demonstrando a baixa qualidade de proteção. Fora esse detalhe, o produto cumpre o prometido.
'''
)

AnalyzerOutput(output=NEU, probas={NEU: 0.921, NEG: 0.065, POS: 0.014})

In [10]:
modelo_analise_sentimento.predict('''
Em menos de 1 ano parou de funcionar, enviei para assistência técnica por estar na garantia,
trocaram o motor, passou a ficar menos potente e não durou 2 utilizações.
Isso se repetiu várias vezes, até que desisti de ficar levando lá e queimando de novo em seguida,
vi outros clientes com o mesmo problema. Não comprem!!
'''
)

AnalyzerOutput(output=NEG, probas={NEG: 0.935, NEU: 0.056, POS: 0.009})

## **Aplicando o modelo aos dados**

In [11]:
import pandas as pd

In [12]:
dados = pd.read_csv('https://raw.githubusercontent.com/alura-cursos/hugging_face/main/Dados/resenhas.csv')

In [13]:
print(f'Device selected: {device}')
# Processa as previsões diretamente
# O device já foi configurado na criação do modelo (create_analyzer)

def process_reviews_safe(reviews):
    results = []
    for text in reviews:
        try:
            result = modelo_analise_sentimento.predict(text)
            results.append(result)
        except Exception as e:
            print(f"Error processing text: {e}")
            continue
    return results

# Process reviews one by one
resultados_previsao = process_reviews_safe(dados['Resenha'].tolist())

Device selected: cpu


In [14]:
sentimento = []

for resultado in resultados_previsao:
  sentimento.append(resultado.output)

In [15]:
sentimento

['POS',
 'POS',
 'POS',
 'NEG',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'NEG',
 'NEG',
 'NEG',
 'NEU',
 'NEG',
 'NEG',
 'NEG',
 'NEU',
 'NEG',
 'NEG',
 'NEU',
 'NEG',
 'NEG',
 'NEG',
 'NEG',
 'NEG',
 'NEG',
 'NEG',
 'NEG',
 'NEG',
 'NEG',
 'POS',
 'POS',
 'NEU',
 'POS',
 'POS']

In [16]:
dados['Sentimento'] = sentimento

In [17]:
dados

,ID,Resenha,Sentimento
0,24,"A fritadeira é sensacional, muito além do que ...",POS
1,733,"Após usar o produto, achei-o fácil e muito efi...",POS
2,865,"Muito funcional, prática e moderna.",POS
3,809,"Boa, mas não das melhores pois a frente de vid...",NEG
4,628,Eu comecei a usar e é bem espaçosa. Gostei poi...,POS
5,657,"Usei ela algumas vezes já, muito silenciosa, l...",POS
6,182,"Superou minhas expectativas, por enquanto tudo...",POS
7,815,"Chegou no prazo. Amei, é muito linda. Não vejo...",POS
8,343,"Estou muito satisfeito com a negociação, receb...",POS
9,624,Lindaaaa amei super indico,POS
